In [1]:
import evennia
evennia._init()
tavis = evennia.search_object('tavis')[0]


In [15]:
from world.utils.db import search_mobdb, search_objdb, search_roomdb, search_zonedb

search_objdb(extra="category religion")

[{'vnum': 1,
  'key': 'friend mortals book',
  'sdesc': 'Book: Friend of All Mortals',
  'ldesc': 'a book titled Friend of All Mortals.',
  'extra': {'language': 'tamrielic', 'category': 'religion', 'date': 'Unknown', 'author': "The Resolute Templar of Ska'vyn", 'title': 'The Friend of All Mortals', 'contents': '"Come to me, Stendarr, for without you, I might be deaf to the manswarm\nmurmurings of thy people, and forgetting their need for comfort and wisdom, I\nmight indulge myself in vain scribblings."\n\nCall him Stendarr, call him Stuhn, call him what you will, but the God of Mercy and Justice is the friend to all the mortals of the Mundus, whether they acknowledge him or not. Yea, even the heretic Dark Elves of Morrowind may use his magic of defense and healing, even so the scaled folk of Argonia, for Stendarr in his benevolence draws no distinction between those who rightfully worship him and those who, in their ignorance and error, do not.\n\nJoin with me, then, in a prayer to St

In [ ]:
import re
from tinydb import Query, where
from world.utils.db import DB

_RE_COMPARATOR_PATTERN = re.compile(r"(<[>=]?|>=?|!)")
# criteria = {'key': 'mora', 'level': '>=150', 'flags': ["nosleep","nosummon"]}# 'extra': 'language aldmerish'}
criteria = {'level': '200-250'}
mobdb = DB.table('mobdb')

def like(val, cmp):
    return cmp in val

def between(val, l, h):
    return l <= val <=h

def make_query(key, value):
    if key == 'extra':
        k,v = value.split(' ')
        return Query()[key][k] == v
    
    if isinstance(value, str):
        matches = re.split(_RE_COMPARATOR_PATTERN, value)
        if matches and len(matches) > 1:
            matches = matches[1:]
            condition, value = matches
            value = int(value)
            if condition == ">=":
                return Query()[key] >= value
            elif condition == ">":
                return Query()[key] > value
            elif condition == "<=":
                return Query()[key] <= value
            elif condition == "<":
                return Query()[key] < value
        
        matches = value.split('-')
        if matches and len(matches) == 2:
            # between range
            low, high = matches
            try:
                low, high = int(low), int(high)
            except ValueError:
                return Query()
            
            return Query()[key].test(between,low, high)
            
        return Query()[key].test(like, value)
        
    elif isinstance(value, list):
        return Query()[key].any(value)

query = None
for key, value in criteria.items():  
    if not query:
        query = make_query(key, value)
#         query = Query()[key].test(like, value)
    else:
        query &= make_query(key, value)
#         query &= Query()[key].test(like, value)

records = mobdb.search(query)

# records = mobdb.search(Query()['key'].test(lambda x: 'mora' in x))
[record['vnum'] for record in records]
# print(query)

    

In [ ]:
mobdb.all()[0]

In [ ]:
import pickle
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

from world.utils.db import *
from typeclasses.objs.custom import CUSTOM_OBJS
from typeclasses.mobs.mob import Mob
    
class EntityLoader:
    """
    handels dynamically loading entities, the caller of this class
    should be location
    """
    def __init__(self, caller, parent, children, amount=1):
        self.type, self.vnum = parent.split(' ')
        self.caller= caller
        self.amount = amount
        _children_list = []
        if children:
            for child in children:
                type_, vnum = child.split(' ')
                _children_list.append([type_, vnum])
        self.children = _children_list
        
        
        if self.type == 'mob':
            self.name = search_mobdb(vnum=int(self.vnum))[int(self.vnum)]['sdesc']
        elif self.type == 'obj':
            self.name = search_objdb(vnum=int(self.vnum))[int(self.vnum)]['sdesc']
        else:
            self.name = "not implemented"
    def load(self):
            
        for _ in range(self.amount):
            if self.type == 'mob':
#                 mob = evennia.create_object(Mob, key=self.vnum)
                print(f"{mobdb[int(self.vnum)]['sdesc']}")
                for ctype, cvnum in self.children:
                    if ctype != 'obj':
                        continue
                    obj_bp = search_objdb(cvnum)
                    if not obj_bp:
                        print('obj does not exist, skipping')
                        continue
                    obj_bp = obj_bp[int(cvnum)]
                    print(f"creating obj on mob: {obj_bp['key']}")
#                     obj = evennia.create_object(CUSTOM_OBJS[obj_bp['type']], key=int(cvnum))
#                     obj.move_to(mob, quiet=True)

#                 mob.move_to(self.caller, quiet=True)


            elif self.type == 'obj':
                obj_bp = search_objdb(self.vnum)
                if not obj_bp:
                    print('obj does not exist, skipping')
                    return
                obj_bp = obj_bp[int(self.vnum)]
                print(f"creating obj in room: {obj_bp['key']}")
#                 obj = evennia.create_object(CUSTOM_OBJS[obj_bp['type']], key=int(self.vnum))
#                 obj.move_to(self.caller, quiet=True)

    
    def read(caller, yaml):
        data = load(x, Loader=Loader)
        parsed= []
        for parent, children in data.items():
            amount = 1
            obj = parent.split(' ')
            if len(obj) != 2:
                print(f'error parsing {parent}')
                continue
            if children:
                for child in children:
                    obj = child.split(' ')
                    if len(obj) !=2 :
                        print(f"error parsing child of {parent}")
                        continue
                    if obj[0] == 'num':
                        amount = int(obj[1])
#             parsed[parent] = children
            parsed.append(EntityLoader(caller, parent, children, amount=amount))

        return parsed
        
x = """
mob 2:
- obj 1

mob 3:
- num 5
obj 11:
"""
split = [t.strip() for t in x.split('\n') if t and 'num' not in t]
print(split)
data = load(x, Loader=Loader)
for x in EntityLoader.read(None, x):
    print(x.name)

#     x.load()

### Mob Difficulty Calculations for HP, MP, SP

main code used to experiment as well as act as a guideline to calculate
all vital attributes

In [ ]:
import numpy as np
import math
from enum import IntEnum

import matplotlib.pyplot as plt
from evennia.contrib.dice import roll_dice

tavis = evennia.search_object('tavis')[0]

def avg_roll(num, size, mod=0):
    #( ( Max Die Roll + 1 ) / 2 ) * Number of Same-Sided Dice
    return int((((size+1)/2)*num)+mod)

class ndict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for k,v in self.items():
            setattr(self, k, v)
    def __setattr__(self, k, v):
        super().__setattr__(k,v)
        self[k] = v
    
    def __getattr__(self, k):
        return self[k]

def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)
    
class MobDifficulty(IntEnum):
    wimpy = 0
    easy = 1
    medium = 2
    hard = 3
    insane = 4
    chaotic = 5

    def members(return_dict=False):
        if return_dict:
            return {
                k.lower(): v
                for k, v, in MobDifficulty._member_map_.items()
            }
        return list(reversed(MobDifficulty._member_map_.keys()))
    
    def next(self):
        value = self.value + 1
        if value > MobDifficulty.chaotic.value:
            return self
        
        return MobDifficulty(value)
    
class AutoMobScaling:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
    def translate(self, value, leftMin, leftMax, rightMin, rightMax):
        # Figure out how 'wide' each range is
        leftSpan = leftMax - leftMin
        rightSpan = rightMax - rightMin

        # Convert the left range into a 0-1 range (float)
        valueScaled = float(value - leftMin) / float(leftSpan)

        # Convert the 0-1 range into a value in the right range.
        return rightMin + (valueScaled * rightSpan)
    @property
    def base(self):
        return (self.base_mult * np.log2(self.olvl)) * self.olvl
    @property
    def diff_mod(self):
        return self.diff_mult ** (self.diff.value-1)
    
    def __repr__(self):
        return str(self.calc())

    def calc_hp(self):
        val = int((self.base * np.log(self.end)) * self.diff_mod)
        return max(val, 3)

    def calc_mp(self):
        val = int((self.base * np.log(
            (self.int + self.wp) / 2)) * self.diff_mod)
        return max(val, 3)

    def calc_sp(self):
        val = int((self.base * np.log(
            (self.end + self.agi) / 2)) * self.diff_mod)
        return max(val, 3)
    
    def calc_dam(self):
        """returns tuple of dice roller
        num, size, mod, (avg)
        ex: [num]D[size] + mod
        """
        old_min = MobDifficulty.wimpy.value
        old_max = MobDifficulty.chaotic.value
        avg = self.calc_hp() / self.translate(
            self.diff.value, old_min, old_max, old_max + 1, old_min + 1)
        
        num = max(1, int(avg * 1.1))
        vals = []
        threshold = 5
        while not vals:
            for x in range(1, 200):  # x
                for y in range(1, 200):  # y
                    tavg = int((((y + 1) / 2) * x))
                    if abs(tavg - num) < threshold:
                        vals.append((x, y))
            threshold += 10
        v = None
        mwp = len(vals) // 2
        if vals:
            v = vals[mwp]
        if v:  
            dam_num = v[0]
            dam_size = v[1]
            dam_max = int(dam_num * dam_size)
            return dam_max, int(avg_roll(dam_num, dam_size)), (round(avg_roll(dam_num, dam_size)/self.calc_hp(), 2))
        return 0,0

    def calc_hit(self):
        hit_roll = min(
            99,
            int(self.translate(self.olvl, 1, 250, 40, 100)) +
            int(self.translate(self.diff.value, 0, 5, -5, 25)))
        return hit_roll

    
stat_names = ['str', 'end','agi', 'int', 'wp','prc', 'prs', 'lck']
diff_mod = 2
diffs = ('wimpy', 'easy', 'medium', 'hard','insane')

m = ndict(
olvl = 1,
base_mult = 1.5,
base_stat = 6,
num_dice = 2,
size_dice = 5,
diff = MobDifficulty.wimpy,
diff_mult=2,
)

LEVELS = 250
BATCH = 25
SAMPLES = (LEVELS, BATCH)
LEVEL_START = 1

dice = np.vectorize(lambda x: x + roll_dice(m.num_dice, m.size_dice))
stats = dice(np.full((1,8), fill_value=m.base_stat, dtype=np.float64))
stats = np.tile(stats, (SAMPLES[0],1))

for _idx, diff in enumerate(diffs):
    base = stats

    results = []
    print(f"{m.diff.name.capitalize()} {m.diff_mult ** m.diff.value}x")
    
    for level, data in enumerate(base):
        if (level+LEVEL_START) % SAMPLES[1] == 0 or (level) == 0:
            n = ndict({**m, **ndict(zip(stat_names, data))})
            n.olvl = level+LEVEL_START
            auto = AutoMobScaling(**n)
            results.append([n.olvl, auto.calc_hp(), auto.calc_mp(), auto.calc_sp(), auto.calc_dam()]+data.tolist())
            
    m.diff=m.diff.next()

    from tabulate import tabulate
    print(tabulate(results,headers=['hp','wp', 'sp', 'dam']+stat_names)+'\n')
#     break


### Experimentation with calcing dam_roll statistics based on level

The base assumption.

If two equal level mobs are fighting with same hp, wp, sp and doing nothing special. Their should be a 50/50 chance the other wins in 5 rounds. From this baseline, we can calculate the potential dam_roll and hit based on level

Each tier of difficulty of the above situation falls in favor on the mob by a 15% difference.. 

Chance of Winning for Player

| Difficulty | Player |  Mob
|------------|--------|-----
|  wimpy     |  90    | 10
|  easy      |  50    | 50
|  medium    |  40    | 60
|  hard      |  20    | 80
|  insane    |  10    | 90
|  chaotic   |   5    | 95


In [ ]:
def bench(samples, num, size, mod=0):
    x = [roll_dice(num, size, ('+', mod)) for _ in range(samples)]
    return np.array(x).mean()

def test(val, diff):
    avg = val/translate(diff, 0,5,6,1)
    num = avg/5
    size = 9.0
    print(num, size, avg)
    return int(bench(1000, num,size))

test(558, 1)
